## Tabular 데이터 캐글 머신러닝 파이프라인
1. 데이터 전처리
2. 피처 엔지니어링
3. 머신러닝 모델 학습
4. 테스트 데이터 예측 및 캐글 업로드

## 1. 데이터 전처리
- 제품 변수 결측값 → 0으로 대체   
제품 보유 여부에 대한 정보가 없을 경우 보유하고 있지 않다고 가정
- 훈련 데이터와 테스트 데이터 통합
- 범주형, 수치형 데이터 전처리   
범주형 데이터 → .factorize() → 라벨 인코딩   
object 타입 수치형 데이터 → .unique() → 특이값 대체 또는 제거 후 정수형 변환
- 모델 학습에 사용할 변수 이름 → features 리스트에 미리 담기

### 1-1. 데이터 준비

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [4]:
np.random.seed(2018)

In [3]:
# 데이터 로드
trn = pd.read_csv('/content/drive/MyDrive/kaggle/santander/data/train_ver2.csv')
tst = pd.read_csv('/content/drive/MyDrive/kaggle/santander/data/test_ver2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1-2. 데이터 전처리

In [4]:
# 제품 변수 별도 저장
prods = trn.columns[24:].tolist()

In [5]:
# 결측치 → 0으로 대체
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

In [6]:
# 24개 제품 중 하나도 보유하지 않는 고객 데이터 제거
no_product = trn[prods].sum(axis = 1) == 0
trn = trn[~no_product]

In [7]:
# 훈련 데이터 + 테스트 데이터
for col in trn.columns[24:]:
  tst[col] = 0
df = pd.concat([trn, tst], axis = 0)

In [8]:
# 학습에 사용할 변수 담을 list 생성
features = []

In [9]:
# 범주형 변수 → .factorize() → 라벨 인코딩
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
  df[col], _ = df[col].factorize(na_sentinel = -99)

features += categorical_cols

In [12]:
# 수치형 변수 특이값, 결측값 → -99로 대체, 정수형 변환
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [10]:
# 학습에 사용할 수치형 변수를 features에 담기
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

## 2. 피처 엔지니어링
- 고객 변수 24개, 날짜 변수 기반 파생 변수 4개, lag-1 변수 24개
- fecha_alta(첫 계약 날짜), ult_fec_cli_1t(마지막으로 1등급이었던 날짜) → 연도, 월 정보 추출
- 결측값 → -99로 임시 대체 (xgboost의 경우 결측값 사용이 가능하나, 일단 임시 대체)

In [11]:
# 두 날짜 변수 → 연도, 월 정보 추출
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)

features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)

features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [15]:
# 결측값 → -99 대체
df.fillna(-99, inplace = True)

In [16]:
# 날짜를 숫자로 변환하는 함수. 2015-01-28은 1, 2016-06-28은 18로 변환
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

In [17]:
# 날짜를 숫자로 변환하여 int_date에 저장
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [16]:
# 데이터 복사, int_date 날짜에 1을 더하여 lag 생성. 변수명에 _prev를 추가.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

df와 df_lag를 병합하는 과정에서 메모리 사용량이 너무 많아 두 데이터프레임을 따로 저장

In [17]:
# df.to_csv('/content/drive/MyDrive/kaggle/santander/data/train.csv')

In [18]:
# df_lag.to_csv('/content/drive/MyDrive/kaggle/santander/data/train_lag.csv')

런타임 초기화 후 다시 시작한 뒤, 저장해 놓은 df를 다시 불러옴

In [19]:
# df = pd.read_csv('/content/drive/MyDrive/kaggle/santander/data/train.csv')

df_lag의 경우, chunksize를 지정해 분할하여 불러온 뒤 차례대로 df와 병합해주는 함수 생성

In [18]:
def preprocess(x):
  df_trn = pd.merge(df, x, on = ['ncodpers', 'int_date'], how = 'left')
  df_trn.to_csv('/content/drive/MyDrive/kaggle/santander/data/df_trn.csv', mode = 'a', header = 'False', index = 'False')

In [ ]:
reader = pd.read_csv('/content/drive/MyDrive/kaggle/santander/data/train_lag.csv', chunksize = 200000)

for r in reader:
  preprocess(r)

(너ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ무 오래걸린다)

In [ ]:
# 메모리 효율을 위한 불필요한 변수 메모리에서 제거
# del df, reader
del df

In [ ]:
# 저번 달 제품 정보 존재 X → 0으로 대체
for prod in prods:
  prev = prod + '_prev'
  df_trn[prev].fillna(0, inplace = True)
df_trn.fillna(-99, inplace = True)

In [ ]:
# lag-1 변수 추가
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

In [ ]:
####
# 피처 엔지니어링 추가 부분
####

## 3. 머신러닝 모델 학습

### 3-1. 교차 검증


캐글 : 하루에 최대 5개의 예측 결과물을 캐글에 제출 할 수 있음   
→ 최대한 올바른 교차 검증 과정을 거친 후에 성능 개선 여부를 확인하는 것이 매우 중요   
   
최신 데이터를 검증 데이터로 분리, 나머지를 훈련 데이터로 사용하여 검증하는 것이 일반적   

In [ ]:
## 모델 학습
# 데이터 → 훈련, 테스트용으로 분리
# 훈련: 2016/01/28 ~ 2016/04/28, 검증: 2016/05/28
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28'] # 기간 설정
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)] # 훈련 데이터
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28'] # 검증 데이터
del df_trn

In [ ]:
# 훈련 데이터 중 신규 구매 건수만 추출
X = []
Y = []
for i, prod in enumerate(prods):
  prev = prod + '_prev'
  prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
  prY = np.zeros(prX.shape[0], dtype = np.int8) + i
  X.append(prX)
  Y.append(prY)

XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [ ]:
# 데이터 분리
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

### 3-2. 모델

- XGBoost 모델 사용 : 대부분의 캐글 상위 입상자들이 사용하는 모델
- XGBoost에서 주로 사용되는 파라미터   
1) max_depth: 트리 모델의 최대 깊이   
2) eta: 딥러닝에서의 learning rate와 같은 개념. 0과 1 사이의 값. 너무 높으면 학습이 잘 되지 않고, 너무 낮으면 학습이 느릴 수 있음   
3) colsample_bytree: 트리 생성 시 훈련 데이터에서 변수를 샘플링해주는 비율. 보통 0.6 ~ 0.9 사용   
4) colsample_bylevel: 트리의 레벨 별로 훈련 데이터의 변수를 샘플링해주는 비율. 보통 0.6 ~ 0.9 사용
- 엄청난 수준의 피처 엔지니어링 + 적당한 파라미터 튜닝 >>> 적당한 피처 엔지니어링 + 엄청난 수준의 파라미터 튜닝

In [ ]:
## 모델링 과정
# 파라미터 설정
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

In [ ]:
# 훈련, 검증 데이터 → XGBoost 형태로 변환
X_trn = XY_trn.as_matrix(columns = features)
Y_trn = XY_trn.as_matrix(columns = ['y'])
dtrn = xgb.DMatrix(X_trn, label = Y_trn, feature_names = features)

X_vld = XY_vld.as_matrix(columns = features)
Y_vld = XY_vld.as_matrix(columns = ['y'])
dvld = xgb.DMatrix(X_vld, label = Y_vld, feature_names = features)

In [ ]:
# 학습
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round = 1000, evals = watch_list, early_stopping_rounds = 20)

In [ ]:
# 학습한 모델 저장
import pickle

pickle.dump(model, open('/content/drive/MyDrive/kaggle/santander/data/xgb.baseline.pkl', 'wb'))
best_ntree_limit = model.best_ntree_limit

### 3-3. 교차 검증

평가척도인 MAP@7을 사용하여 성능 수준을 확인   
현재 베이스라인 코드에서 받을 수 있는 최고 점수 = 0.042663

In [ ]:
## MAP@7 평가 척도를 위한 준비 작업
# 고객 식별 번호 추출
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns = ['ncodpers'])

In [ ]:
# 검증 데이터에서 신규 구매 구하기
for prod in prods:
  prev = prod + '_prev'
  padd = prod + '_add'
  vld[padd] = vld[prod] - vld[prev]

add_vld = vld.as_matrix(columns = [prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

In [ ]:
# 고객별 신규 구매 정답값 → add_vld_list에 저장, 총 count → count_vld에 저장
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
  for prod in range(len(prods)):
    if add_vld[ncodper, prod] > 0:
      add_vld_list[ncodper].append(prod)
      count_vld += 1

In [ ]:
# 현재 얻을 수 있는 MAP@7 최고점을 미리 계산
print(mapk(add_vld_list, add_vld_list, 7, 0.0)) # mapk 함수 사용

In [ ]:
# 검증 데이터에 대한 예측 값 계산
X_vld = vld.as_matrix(columns = features)
Y_vld = vld.as_matrix(columns = ['y'])
dvld = xgb.DMatrix(X_vld, label = Y_vld, feature_names = features)
preds_vld = model.predict(dvld, ntree_limit = best_ntree_limit)

In [ ]:
# 저번 달 보유 제품 → 신규 구매 불가능 → 확률값에서 미리 1을 빼줌
preds_vld = preds_vld - vld.as_matrix(columns = [prod + '_prev' for prod in prods])

In [ ]:
# 검증 데이터 예측 상위 7개 추출
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
  y_prods = [(y, p, ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
  y_prods = sorted(y_prods, key = lambda a: a[0], reverse = True)[:7]
  result_vld.append([ip for y, p, ip in y_prods])

In [ ]:
# 검증 데이터에서의 점수 추출
print(mapk(add_vld_list, result_vld, 7, 0.0)) # 0.036466

## 4. 테스트 데이터 예측 및 캐글 업로드


In [ ]:
# 전체 훈련 데이터로 재학습
X_all = XY.as_matrix(columns = features)
Y_all = XY.as_matrix(columns = ['y'])
dall = xgb.DMatrix(X_all, label = Y_all, feature_names = features)
watch_list = [(dall, 'train')]

In [ ]:
# 트리 개수 → 늘어난 양만큼 비례해서 증가
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))

In [ ]:
# 모델 재학습
model = xgb.train(param, dall, num_boost_round = best_ntree_limit, evals = watch_list)

In [ ]:
# 변수 중요도 출력
print('Featrue importance')
for kv in sorted([(k, v) for k, v in model.get_fscore().items()], key = lambda kv:kv[1], reverse = True):
  print(kv)

In [ ]:
# 테스트 데이터에 대한 예측 값 계산
X_tst = tst.as_matrix(columns = features)
dtst = xgb.DMatrix(X_test, feature_names = features)
preds_tst = model.predict(dtst, ntree_limit = best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns = ['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns = [prod + '_prev' for prod in prods])

In [ ]:
# 제출 파일 생성
submit_file = open('/content/drive/MyDrive/kaggle/santander/data/xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers, added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
  y_prods = [(y, p, ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
  y_prods = [p for y, p, ip in y_prods]
  submit_file.write('{}, {}\n'.format(int(ncodper), ' '.join(y_prods)))